In [192]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, mean_absolute_error

In [193]:
dx = pd.read_csv("training_set_features.csv", index_col="respondent_id")
dy = pd.read_csv("training_set_labels.csv")
dy_test = pd.read_csv("test_set_features.csv", index_col="respondent_id")
dy_xyz = dy["xyz_vaccine"]
dy_seasonal = dy["seasonal_vaccine"]
dx

,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_xyz,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
respondent_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,qufhixun,Non-MSA,0.0,0.0,NaN,NaN
26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,"<= $75,000, Above Poverty",Not Married,Rent,Employed,lzgpxyit,"MSA, Principle City",1.0,0.0,fcxhlnwr,cmhcxjea
26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,...,NaN,Not Married,Own,NaN,lzgpxyit,"MSA, Not Principle City",0.0,0.0,NaN,NaN


In [194]:
dx.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26707 entries, 0 to 26706
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   xyz_concern                  26615 non-null  float64
 1   xyz_knowledge                26591 non-null  float64
 2   behavioral_antiviral_meds    26636 non-null  float64
 3   behavioral_avoidance         26499 non-null  float64
 4   behavioral_face_mask         26688 non-null  float64
 5   behavioral_wash_hands        26665 non-null  float64
 6   behavioral_large_gatherings  26620 non-null  float64
 7   behavioral_outside_home      26625 non-null  float64
 8   behavioral_touch_face        26579 non-null  float64
 9   doctor_recc_xyz              24547 non-null  float64
 10  doctor_recc_seasonal         24547 non-null  float64
 11  chronic_med_condition        25736 non-null  float64
 12  child_under_6_months         25887 non-null  float64
 13  health_worker        

In [195]:
dx.isnull().sum()

xyz_concern                       92
xyz_knowledge                    116
behavioral_antiviral_meds         71
behavioral_avoidance             208
behavioral_face_mask              19
behavioral_wash_hands             42
behavioral_large_gatherings       87
behavioral_outside_home           82
behavioral_touch_face            128
doctor_recc_xyz                 2160
doctor_recc_seasonal            2160
chronic_med_condition            971
child_under_6_months             820
health_worker                    804
health_insurance               12274
opinion_xyz_vacc_effective       391
opinion_xyz_risk                 388
opinion_xyz_sick_from_vacc       395
opinion_seas_vacc_effective      462
opinion_seas_risk                514
opinion_seas_sick_from_vacc      537
age_group                          0
education                       1407
race                               0
sex                                0
income_poverty                  4423
marital_status                  1408
r

In [196]:
categories = dx.columns
categories

Index(['xyz_concern', 'xyz_knowledge', 'behavioral_antiviral_meds',
       'behavioral_avoidance', 'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_xyz', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'health_insurance', 'opinion_xyz_vacc_effective', 'opinion_xyz_risk',
       'opinion_xyz_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group',
       'education', 'race', 'sex', 'income_poverty', 'marital_status',
       'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa',
       'household_adults', 'household_children', 'employment_industry',
       'employment_occupation'],
      dtype='object')

In [197]:
for i in categories:
        cat_mode = dx[i].mode()[0]
        dx[i].fillna(cat_mode, inplace=True)

for i in categories:
        cat_mode = dy_test[i].mode()[0]
        dy_test[i].fillna(cat_mode, inplace=True)



dx.isnull().sum()

C:\Users\91909\AppData\Local\Temp\ipykernel_41304\66204306.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dx[i].fillna(cat_mode, inplace=True)
C:\Users\91909\AppData\Local\Temp\ipykernel_41304\66204306.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing

xyz_concern                    0
xyz_knowledge                  0
behavioral_antiviral_meds      0
behavioral_avoidance           0
behavioral_face_mask           0
behavioral_wash_hands          0
behavioral_large_gatherings    0
behavioral_outside_home        0
behavioral_touch_face          0
doctor_recc_xyz                0
doctor_recc_seasonal           0
chronic_med_condition          0
child_under_6_months           0
health_worker                  0
health_insurance               0
opinion_xyz_vacc_effective     0
opinion_xyz_risk               0
opinion_xyz_sick_from_vacc     0
opinion_seas_vacc_effective    0
opinion_seas_risk              0
opinion_seas_sick_from_vacc    0
age_group                      0
education                      0
race                           0
sex                            0
income_poverty                 0
marital_status                 0
rent_or_own                    0
employment_status              0
hhs_geo_region                 0
census_msa

In [198]:
for i in categories:
        dx[i] = dx[i].astype("category").cat.codes
        
for i in categories:
        dy_test[i] = dy_test[i].astype("category").cat.codes
dx

,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_xyz,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
respondent_id,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,0,0,0,0,1,1,0,...,2,1,0,1,8,2,0,0,4,21
1,3,2,0,1,0,1,0,1,1,0,...,2,1,1,0,1,0,0,0,12,19
2,1,1,0,1,0,0,0,0,0,0,...,0,1,0,0,9,0,2,0,14,21
3,1,1,0,1,0,1,1,0,0,0,...,2,1,1,1,5,1,0,0,4,21
4,2,1,0,1,0,1,1,0,1,0,...,0,0,0,0,9,0,1,0,18,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,2,0,0,1,0,0,0,1,0,0,...,0,1,0,1,9,2,0,0,4,21
26703,1,2,0,1,0,1,0,0,0,1,...,0,1,1,0,6,1,1,0,4,2
26704,2,2,0,1,1,1,1,0,1,0,...,0,1,0,0,6,0,0,0,4,21


In [199]:
dx_xyz = dx#.drop(["doctor_recc_seasonal", "opinion_seas_vacc_effective", "opinion_seas_risk", "opinion_seas_sick_from_vacc"],axis=1)
dx_seasonal = dx#.drop(["doctor_recc_xyz", "opinion_xyz_vacc_effective", "opinion_xyz_risk", "opinion_xyz_sick_from_vacc", "xyz_concern", "xyz_knowledge"],axis=1)

In [200]:
x_xyz_train, x_xyz_test, y_xyz_train, y_xyz_test = train_test_split(dx_xyz, dy_xyz, test_size=0.05)
x_seas_train, x_seas_test, y_seas_train, y_seas_test = train_test_split(dx_seasonal, dy_seasonal, test_size=0.05)

model_xyz = RandomForestClassifier()
model_seas = RandomForestClassifier()

model_seas.fit(x_seas_train, y_seas_train)


RandomForestClassifier()

In [201]:
model_xyz.fit(x_xyz_train, y_xyz_train)

RandomForestClassifier()

In [212]:
predict_xyz = model_xyz.predict_proba(x_xyz_test)[:, 1]
predict_xyz

array([0.11, 0.24, 0.09, ..., 0.22, 0.3 , 0.51])

In [213]:
predict_seas = model_seas.predict_proba(x_seas_test)[:, 1]
predict_seas

array([0.43, 0.39, 0.11, ..., 0.87, 0.33, 0.84])

In [214]:
roc_auc_score(y_seas_test, predict_seas)

0.84045142978943

In [215]:
roc_auc_score(y_xyz_test, predict_xyz)

0.8066887431505371

In [216]:
model_xyz.score(x_xyz_test, y_xyz_test)

0.8413173652694611

In [217]:
model_seas.score(x_seas_test, y_seas_test)

0.7627245508982036